<a href='http://www.holoviews.org'><img src="../../assets/pn_hv_gv_bk_ds_pa.png" alt="HoloViz logos" width="40%;" align="left"/></a>
<div style="float:right;"><h2>Exercise: Building a Dashboard</h2></div>

<div class="alert alert-warning" role="alert"> <strong>WORK IN PROGRESS:</strong> We are in the progress of updating these materials in anticipation of a tutorial at the 2019 SciPy conference. Work will be complete by the morning of July 8th 2019. Check out <a href="https://github.com/pyviz/holoviz/tree/v0.1.1">this tag</a> to access the materials as they were before these changes started. For the latest version of the tutorial, visit <a href="https://holoviz.org/tutorial">holoviz.org</a>.
</div>

In [ ]:
import panel as pn
import holoviews as hv
from holoviews import opts  # noqa

pn.extension('katex')
hv.extension('bokeh')

### Exercise 1

In this exercise you will construct a number of Panel components and then lay them out as a non-interactive Panel dashboard.

#### The data

Throughout this tutorial we will be working with one core dataset, a collection of earthquakes recorded between 2000-2018 provided by the US Geological Survey (USGS). The data is provided as a parquet file as part of the tutorial and we will load it using dask and persist it. We will return to this later, for now we will focus on building a dashboard and you don't know any of the details about the dataset or the dask or pandas API.

In [ ]:
import dask.dataframe as dd

df = dd.read_parquet('../../data/earthquakes.parq')

df = df[~df.mag.isna()].persist()

#### The logo

The first component of the dashboard is an image of the US Geological Survey logo, start by declaring a pane containing the logo and assign it to the ``logo`` variable. Also set a width to ensure the logo doesn't appear to big.

In [ ]:
logo_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/cd/USGS_logo.svg/2000px-USGS_logo.svg.png'

## Define a panel component containing the logo
logo = ...

## Display it


<details><summary>Solution</summary><br>

```python
logo = pn.panel(logo_url, width=300) 
```

</details>

#### Richter scale equation

Next we will create a component to display the equation for the Richter scale definition. Declare the appropriate pane and assign it to the ``equation`` variable.

In [ ]:
equation_string = '$M_L = log_{10}A - log_{10} A_0(\delta)$'

## Define a panel component containing the equation (Hint: Use the LaTeX pane)
equation = ...

## Display it


<details><summary>Solution</summary><br>

```python
logo = pn.pane.LaTeX(equation_string) 
```

</details>

#### Computing statistics

In [ ]:
year = 2000

def yearly_stats_fn(year):
    return df[df.time.dt.year == year][['mag', 'depth', 'rms']].describe().compute()

## Create a panel component by calling the function with a particular year
yearly_stats = ...

## Display it


<details><summary>Solution</summary><br>

```python
yearly_stats = pn.panel(stats_df) 
```

</details>

#### List the strongest earthquakes

In [ ]:
def strongest_earthquakes_fn(year):
    year_df = df[df.time.dt.year == year].compute()
    return year_df.sort_values('mag', ascending=False).iloc[:5][['time', 'place', 'mag']].reset_index(drop=True)

## Create a panel component by calling the function with a particular year
strongest_earthquakes = ...

## Display it


<details><summary>Solution</summary><br>

```python
strongest_earthquakes = pn.panel(strongest_df) 
```

</details>

#### A plot

If you are up to it create a custom plot from the ``year_df`` dataframe defined below, create a panel component and assign it to the ``plot`` variable.



<details><summary>Hint</summary><br>

If you are using matplotlib pyplot you can get the figure with ``plot = plt.gcf()`` and the close it with ``plot.close()``

</details>

In [ ]:
year_df = df[df.time.dt.year == year].compute()

## Create a plot and assign it to the plot variable
plot = ...

## Display it


<details><summary>Solution</summary><br>

```python
plot = hv.BoxWhisker(year_df, 'type', 'mag')
```

</details>

### Composing the dashboard

Now that we have defined all the different components, it is time to lay them out into the overall dashboard.

Arrange the following components into a dashboard using the ``Row`` and ``Column`` panels:

* ``logo``
* ``equation``
* ``yearly_stats``
* ``strongest_earthquakes``
* ``plot`` (optional)

<details><summary>Solution</summary><br>

```python
year = 2000
    
logo = pn.panel(logo_url, width=200)
equation = pn.pane.LaTeX(equation_string)
yearly_stats = yearly_stats_fn(year)
strongest_earthquakes = strongest_earthquakes_fn(year)
plot = hv.BoxWhisker(year_df, 'type', ('mag', 'Mangitude'))
    
pn.Row(
    pn.Column(logo, pn.panel('## The Richter scale'), equation),
    pn.Column('### Statistics', yearly_stats),
    pn.Column('### Strongest Earthquakes', strongest_earthquakes),
    pn.Column('### Magnitude Plot', plot)
)
```
    
</details>

### Exercise 2

Add widgets to control the dashboard.

#### The widget

Declare an ``IntSlider`` widget with a start value of 2000, end value of 2018, and current value of 2000.

<details><summary>Solution</summary><br>

```python
year_slider = pn.widgets.IntSlider(name='Year', start=2000, end=2018, value=2000)
year_slider
```

</details>

### Exercise 3

Add some visualizations via the `.plot` API to the dashboard.

Start by reading in the data from the parquet file.

<details><summary>Hint</summary><br>

Use tab complete to find the `read_parquet` method in the `dask.dataframe` module (`dd`)

</details>

<details><summary>Solution</summary><br>

```python
import dask.dataframe as dd

df = dd.read_parquet('../../data/earthquakes.parq')
```

<br></details>

Make a datashaded scatter of all the earthquakes. Call this plot `earthquake_scatter`.

<details><summary>Hint</summary><br>

Don't forget to include `datashade=True` and to import `hvplot.dask`

</details>

<details><summary>Solution</summary><br>
    
```python
import hvplot.dask
    
earthquake_scatter = df.hvplot.scatter(x='longitude', y='latitude', datashade=True)
earthquake_scatter
```
    
<br></details>

Include this plot in the panel layout that you build in Exercises 1 and 2.

<details><summary>Hint</summary><br>

The panel object is indexable just like a list or a DataFrame, so just replace the item in. Call `print(dashboard)` to inspect the hierarchy

</details>

<details><summary>Solution</summary><br>

```python
dashboard[0] = earthquake_scatter
dashboard
```

<br></details>

You should now have a datashaded scatter plot that shows up in your dashboard.

<details><summary>I don't</summary><br>

```python
import dask.dataframe as dd
import hvplot.dask

earthquake_scatter = df.hvplot.scatter(x='longitude', y='latitude', datashade=True)
dashboard = pn.Column('## Earthquakes', earthquake_scatter)
dashboard
```

<br></details>

### Exercise 4

Add a linked visualization with holoviews.

### Exercise 5

Add a datashaded visualization to the dashboard.